In [171]:
import pandas as pd
import numpy as np
from pydataset import data
import env
import os
import new_lib as nl

In [38]:
users = pd.DataFrame({
    'id': [1,2,3,4,5,6],
    'name': ['Bob', 'Joe', 'Adam', 'Sally', 'Jorge', 'Mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]

})
users

,id,name,role_id
0,1,Bob,1.0
1,2,Joe,2.0
2,3,Adam,3.0
3,4,Sally,3.0
4,5,Jorge,NaN
5,6,Mike,NaN


In [39]:
roles = pd.DataFrame({
    'id': [1,2,3,4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
    
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


In [40]:
users.merge(roles, 
            left_on = 'role_id', 
            right_on = 'id', 
            how = 'right', 
            indicator = True)

# It only shows values that have a role due to the right join

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,Bob,1.0,1,admin,both
1,2.0,Joe,2.0,2,author,both
2,3.0,Adam,3.0,3,reviewer,both
3,4.0,Sally,3.0,3,reviewer,both
4,NaN,NaN,NaN,4,commenter,right_only


In [41]:
users.merge(roles, 
            left_on = 'role_id', 
            right_on = 'id', 
            how = 'outer', 
            indicator = True)

# An outer join will show everything

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,Bob,1.0,1.0,admin,both
1,2.0,Joe,2.0,2.0,author,both
2,3.0,Adam,3.0,3.0,reviewer,both
3,4.0,Sally,3.0,3.0,reviewer,both
4,5.0,Jorge,NaN,NaN,NaN,left_only
5,6.0,Mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


In [42]:
users.drop(columns = 'role_id')

,id,name
0,1,Bob
1,2,Joe
2,3,Adam
3,4,Sally
4,5,Jorge
5,6,Mike


In [43]:
users.merge(roles)

# It will no longer merge due to having no foreign keys

,id,name,role_id


In [44]:
mpg = data('mpg')

In [45]:
data('mpg', show_doc = True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




In [46]:
mpg.shape

#234 rows 11 columns

(234, 11)

In [47]:
mpg.columns

Index(['manufacturer', 'model', 'displ', 'year', 'cyl', 'trans', 'drv', 'cty',
       'hwy', 'fl', 'class'],
      dtype='object')

In [48]:
mpg.describe()

,displ,year,cyl,cty,hwy
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


In [49]:
len(mpg.manufacturer.unique())

15

In [50]:
len(mpg.model.unique())

38

In [51]:
mpg['mileage_difference'] = mpg.hwy - mpg.cty

In [52]:
mpg['mileage_average'] = (mpg.hwy + mpg.cty)/2

In [53]:
mpg['is_automatic'] = np.where(mpg.trans.str.contains('auto'), 'automatic', 'manual')

In [54]:
(
    
    mpg[['manufacturer', 'mileage_average']]
    .groupby('manufacturer')
    .agg('mean')
    .sort_values('mileage_average', ascending = False)
    .head()

)

# Honda has the best mpg

,mileage_average
manufacturer,
honda,28.500000
volkswagen,25.074074
hyundai,22.750000
subaru,22.428571
audi,22.027778


In [55]:
mpg[['is_automatic', 'mileage_average']].groupby('is_automatic').agg('mean').sort_values('mileage_average')

# Manual has the best mpg

,mileage_average
is_automatic,
automatic,19.130573
manual,22.227273


In [56]:
def get_db_url(db, env_file=os.path.exists('env.py')):
    '''
    return a formatted string containing username, password, host and database
    for connecting to the mySQL server
    and the database indicated
    env_file checks to see if the env.py exists in cwd
    '''
    if env_file:
        username, password, host = (env.username, env.password, env.host)
        return f'mysql+pymysql://{username}:{password}@{host}/{db}'
    else:
        return 'You need a username and password'
url = get_db_url('employees')

In [57]:
query = 'SELECT * FROM employees LIMIT 10'
df = pd.read_sql(query, url)
df

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
5,10006,1953-04-20,Anneke,Preusig,F,1989-06-02
6,10007,1957-05-23,Tzvetan,Zielinski,F,1989-02-10
7,10008,1958-02-19,Saniya,Kalloufi,M,1994-09-15
8,10009,1952-04-19,Sumant,Peac,F,1985-02-18
9,10010,1963-06-01,Duangkaew,Piveteau,F,1989-08-24


In [179]:
df.to_csv(path_or_buf = 'test_csv')

In [58]:
# intentionally making typos gives errors about plugin and 
# if I make a SQL error it will tell me that the syntax is wrong

In [59]:
equery = 'SELECT * FROM employees'
emp = pd.read_sql(equery, url)
emp.shape
#300024 row, 6 columns as expected

(300024, 6)

In [60]:
tquery = 'SELECT * FROM titles'
tlt = pd.read_sql(tquery, url)
tlt.shape
#443308 rows, 4 columns as expected more or less

(443308, 4)

In [61]:
emp.describe()

,emp_no
count,300024.000000
mean,253321.763392
std,161828.235540
min,10001.000000
25%,85006.750000
50%,249987.500000
75%,424993.250000
max,499999.000000


In [62]:
tlt.describe()

,emp_no
count,443308.000000
mean,253075.034430
std,161853.292613
min,10001.000000
25%,84855.750000
50%,249847.500000
75%,424891.250000
max,499999.000000


In [63]:
len(tlt.title.unique())

7

In [64]:
tlt.to_date.sort_values().head()

16064     1985-03-01
406358    1985-03-11
346216    1985-03-20
35527     1985-03-23
277603    1985-04-17
Name: to_date, dtype: object

In [65]:
tlt.to_date.sort_values().tail()

191714    9999-01-01
191712    9999-01-01
191707    9999-01-01
191723    9999-01-01
443307    9999-01-01
Name: to_date, dtype: object

In [66]:
urlc = get_db_url('chipotle')

In [67]:
query = 'SELECT * FROM orders'

In [68]:
orders = pd.read_sql(query, urlc)
orders

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39
1,2,1,1,Izze,[Clementine],$3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98
...,...,...,...,...,...,...
4617,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75
4618,4619,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75
4619,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Guacamole, Lettuce]]",$11.25
4620,4621,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettuce]]",$8.75


In [69]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  4622 non-null   int64 
 1   order_id            4622 non-null   int64 
 2   quantity            4622 non-null   int64 
 3   item_name           4622 non-null   object
 4   choice_description  4622 non-null   object
 5   item_price          4622 non-null   object
dtypes: int64(3), object(3)
memory usage: 216.8+ KB


def clean(x):
    x = x.replace('$', '')
    x = x.replace(',', '')
    x = round(float(x), 2)
    return x
    
clean function but I imported from custom library new_lib

In [175]:
help(nl.clean)

Help on function clean in module new_lib:

clean(x)
    Takes a currency considered an int and turns it into a clean float
    rounded to two decimal places



In [172]:
orders['clean_price'] = orders.item_price.apply(nl.clean)

In [173]:
orders[(orders.item_name == 'Chicken Bowl') & (orders.quantity > 1)]

,id,order_id,quantity,item_name,choice_description,item_price,clean_price,total_item_price
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98,16.98,33.96
154,155,70,2,Chicken Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Rice, Lettuce]]",$17.50,17.50,35.00
282,283,124,2,Chicken Bowl,"[Fresh Tomato Salsa, [Rice, Black Beans, Cheese, Sour Cream]]",$17.50,17.50,35.00
409,410,178,3,Chicken Bowl,"[[Fresh Tomato Salsa (Mild), Tomatillo-Green Chili Salsa (Medium), Roasted Chili Corn ...",$32.94,32.94,98.82
415,416,181,2,Chicken Bowl,[Tomatillo Red Chili Salsa],$17.50,17.50,35.00
654,655,271,2,Chicken Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Rice, Lettuce]]",$17.50,17.50,35.00
976,977,401,2,Chicken Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Rice, Lettuce]]",$17.50,17.50,35.00
1017,1018,418,2,Chicken Bowl,"[Fresh Tomato Salsa, [Rice, Cheese, Black Beans]]",$17.50,17.50,35.00
1106,1107,457,2,Chicken Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Rice, Lettuce]]",$17.50,17.50,35.00
1429,1430,578,2,Chicken Bowl,"[Fresh Tomato Salsa, [Rice, Sour Cream, Guacamole]]",$22.50,22.50,45.00


In [73]:
orders['total_item_price'] = orders.clean_price * orders.quantity

In [162]:
orders[['order_id', 'clean_price']].groupby('order_id').agg('sum')

# total price of every order

,clean_price
order_id,
1,11.56
2,16.98
3,12.67
4,21.00
5,13.70
...,...
1830,23.00
1831,12.90
1832,13.20


In [170]:
(
    
    orders[['item_name', 'quantity']]
    .groupby('item_name')
    .agg('sum')
    .sort_values('quantity', ascending = False)
    .head()

)
# chicken bowls are most popular

,quantity
item_name,
Chicken Bowl,761
Chicken Burrito,591
Chips and Guacamole,506
Steak Burrito,386
Canned Soft Drink,351


In [82]:
(
    
    orders[['item_name', 'quantity', 'clean_price']]
    .groupby('item_name')
    .agg('sum')
    .sort_values('clean_price', ascending = False)
    .head()

)
# chicken bowls also make the most profit

,quantity,clean_price
item_name,,
Chicken Bowl,761,7342.73
Chicken Burrito,591,5575.82
Steak Burrito,386,3851.43
Steak Bowl,221,2260.19
Chips and Guacamole,506,2201.04


In [91]:
emtlt = emp.merge(tlt, left_on = 'emp_no', right_on = 'emp_no', how = 'inner')
emtlt

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...,...,...,...,...,...
443303,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Engineer,1987-08-30,1992-08-29
443304,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Senior Engineer,1992-08-29,9999-01-01
443305,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Senior Staff,1998-12-27,9999-01-01
443306,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Staff,1993-12-27,1998-12-27


In [168]:
emtlt[['title', 'hire_date']].groupby('title').max().sort_values('hire_date')

# groupby order doesn't matter what matters is that the agg metric like .agg or .max() immediatly follows the groupby

,hire_date
title,
Manager,1992-02-05
Assistant Engineer,1999-12-24
Technique Leader,1999-12-31
Senior Engineer,2000-01-01
Staff,2000-01-12
Senior Staff,2000-01-13
Engineer,2000-01-28


In [146]:
joins = '''
SELECT t.title, d.dept_name FROM dept_emp AS de
JOIN titles AS t
USING(emp_no)
JOIN departments AS d
USING(dept_no)
WHERE t.to_date >= CURDATE();


'''


jt = pd.read_sql(joins, url)
jt

,title,dept_name
0,Senior Engineer,Development
1,Staff,Sales
2,Senior Engineer,Production
3,Senior Engineer,Production
4,Senior Staff,Human Resources
...,...,...
265327,Engineer,Production
265328,Senior Engineer,Production
265329,Senior Engineer,Development
265330,Senior Staff,Finance


In [147]:
jt['title_as_num']= 1
jt
# Alternate way of doing what was asked but unnecessary if using a crosstable

,title,dept_name,title_as_num
0,Senior Engineer,Development,1
1,Staff,Sales,1
2,Senior Engineer,Production,1
3,Senior Engineer,Production,1
4,Senior Staff,Human Resources,1
...,...,...,...
265327,Engineer,Production,1
265328,Senior Engineer,Production,1
265329,Senior Engineer,Development,1
265330,Senior Staff,Finance,1


In [150]:
cross = pd.crosstab(jt.dept_name, jt.title)
cross

title,Assistant Engineer,Engineer,Manager,Senior Engineer,Senior Staff,Staff,Technique Leader
dept_name,,,,,,,
Customer Service,68,627,1,1790,12349,3902,241
Development,1833,15677,1,43364,1085,315,6117
Finance,0,0,1,0,10650,3199,0
Human Resources,0,0,1,0,10843,3416,0
Marketing,0,0,1,0,12371,3880,0
Production,1542,13325,1,37156,1123,349,5210
Quality Management,433,3744,1,10390,0,0,1422
Research,77,830,1,2250,10219,3206,321
Sales,0,0,1,0,32032,9967,0


In [158]:
cross.sum().sum()
# Using sum more than once will yield the sum of the sum youve taken
# all the way down to the size or actual number of instances in your dataframe

265332

In [149]:
jt[['dept_name', 'title_as_num']].groupby('dept_name').agg('sum')

,title_as_num
dept_name,
Customer Service,18978
Development,68392
Finance,13850
Human Resources,14260
Marketing,16252
Production,58706
Quality Management,15990
Research,16904
Sales,42000
